<a href="https://colab.research.google.com/github/agniji/ONET_Prediction/blob/main/ONET_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ONET JobCode Predictions

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00


In [2]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c57b6c7a725b587a1d038efb441daeb5fb4975217648b890286a92766ebb3fc9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import numpy as np
import pandas as pd
import re

import plotly.express as px
from collections import Counter
from sklearn import metrics
import transformers
from transformers import pipeline
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [4]:
from google.colab import drive

drive.mount("/content/gdrive")

%pwd

Mounted at /content/gdrive


'/content'

In [5]:
#Add shortcut of AgeptID to My Drive
%cd gdrive/My\ Drive/AdeptID

/content/gdrive/.shortcut-targets-by-id/19lWHzir5Uxolv4hNCWgtkyJvG_tj9vxv/AdeptID


In [6]:
from sentence_transformers import SentenceTransformer, util
#from transformers import pipeline

In [7]:
#TRAIN Data
df = pd.read_csv("train_data.csv")
#test data
df_test = pd.read_csv("test_data.csv")

In [8]:
#This module is NOT used due to the long running time, but highly recommended
def summarize(text):

  summarizer = pipeline("summarization")
  #to_tokenize = text-input
  summarized = summarizer(text, min_length=75, max_length=100)

  #print(summarized)
  #The list is converted to a string

  summ=' '.join([str(i) for i in summarized])
  return summ

In [ ]:
#Takes too long
#Avoiding this step, but highly recommended, using a faster interface
#summarize(df['clean_Post'].iloc[0])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


"{'summary_text': ' Executive meeting manager is responsible for ensuring that each group meets or exceeds their revenue expectations based on their contracts . Must possess computer skills including but not limited to use of microsoft word excel and delphi knowledge of sales skills . Must have experience in the field of hospitality with specific experience in catering sales is essential year within the hospitality industry required must have experience at a similar size and quality hotel high school diploma required bachelors degree preferred .'}"

In [87]:
#Truncate the cleaned post to n words, which is close to the model max limit

def trunc(text):

  words = text.split(' ')
  if len(words) > 250:
    new = ' '.join(word for word in  words[:250]) #try w 256, however takes more time
    return new
  else:
    return text


In [74]:
#Clean up the Raw Title & Posting Body;
#Concatenate the two, getting a Clean Job Posting to embed
def clean_text(text):

  res = re.sub(r'(\#\w+)'," ",text)
  res = re.sub(r"br","",res)
  res = re.sub(r'[^\w\s]', ' ', res)
  res = re.sub('[^a-zA-Z]+',' ',res)
  res = res.lower()
  res = res.strip()

  return res


In [11]:
#Clean TRAIN Data
df['clean_Post'] = df['TITLE_RAW'].apply(lambda x : clean_text(x)) + ' ' +  df['BODY'].apply(lambda x : clean_text(x))

#df['Summarized_Post'] = df['clean_Post'].apply(lambda x : summarize(x))
#Truncate Clean_Post to 100 words
df['Truncated_Post'] = df['clean_Post'].apply(lambda x : trunc(x))

In [75]:
#Clean Test Data
df_test['clean_Post'] = df_test['TITLE_RAW'].apply(lambda x : clean_text(x)) + ' ' +  df_test['BODY'].apply(lambda x : clean_text(x))

df_test['Truncated_Post'] = df_test['clean_Post'].apply(lambda x : trunc(x))

In [13]:
df.head(10)

,ID,POSTED,TITLE_RAW,BODY,ONET_NAME,ONET,clean_Post,Truncated_Post
0,3a9bc988d77e46507f6753429dd848a816d0b9b9,2023-05-03,Executive Meeting Manager,Executive Meeting Manager Marriott La Jolla - ...,"Meeting, Convention, and Event Planners",13-1121.00,executive meeting manager executive meeting ma...,executive meeting manager executive meeting ma...
1,eb3a017370d55577e892ff8207a640b7d7136f31,2023-05-03,Rehabilitation Technician-Outpatient Rehab-Fle...,Rehabilitation Technician-Outpatient Rehab-Fle...,Occupational Therapy Aides,31-2012.00,rehabilitation technician outpatient rehab fle...,rehabilitation technician outpatient rehab fle...
2,8717d2213055d39271bd12490263a7fbe603aedb,2023-05-03,Office/Bookkeeping Assistant,"Office/Bookkeeping Assistant\nSanta Barbara, C...","Office Clerks, General",43-9061.00,office bookkeeping assistant office bookkeepin...,office bookkeeping assistant office bookkeepin...
3,43b55e4334835e20e1c64d9ac7bb0a0267369b9e,2023-05-03,Administrative Support Coordinator - VA - (REM...,Find Jobs Administrative Support Coordinator -...,"Secretaries and Administrative Assistants, Exc...",43-6014.00,administrative support coordinator va remote f...,administrative support coordinator va remote f...
4,afa355a328687ddb88d6265a237d0375bb36eae7,2023-05-03,Receptionist/Administrative Assistant,Receptionist/Administrative Assistant Burgess ...,"Secretaries and Administrative Assistants, Exc...",43-6014.00,receptionist administrative assistant receptio...,receptionist administrative assistant receptio...
5,13e3c909e3d707aef02fbbd295d8912d9c70fd6f,2023-05-03,Outsourced Accounting Manager,ZipApply\nOutsourced Accounting Manager\nGloba...,Financial Managers,11-3031.00,outsourced accounting manager zipapply outsour...,outsourced accounting manager zipapply outsour...
6,d83597b0c587a5f51f31f11f18427d4713303060,2023-05-03,Product Owner,You could be the one who changes everything fo...,General and Operations Managers,11-1021.00,product owner you could be the one who changes...,product owner you could be the one who changes...
7,0cca710d39091a0006f45cb381381df4c5bc95a1,2023-05-03,Extended Coverage Team Specialist,Find Jobs Extended Coverage Team Specialist Sw...,Fast Food and Counter Workers,35-3023.00,extended coverage team specialist find jobs ex...,extended coverage team specialist find jobs ex...
8,204d90c58fecaa6b4edada85388fad480188cb48,2023-05-03,Server/Waitstaff,"Server/Waitstaff Our Place Pizzeria Branford, ...",Waiters and Waitresses,35-3031.00,server waitstaff server waitstaff our place pi...,server waitstaff server waitstaff our place pi...
9,7ccb525e00ef91b3c6d68745029e333a0db2882f,2023-05-03,Stockroom Associate,Stockroom Associate HomeGoods (part of The TJX...,"Laborers and Freight, Stock, and Material Move...",53-7062.00,stockroom associate stockroom associate homego...,stockroom associate stockroom associate homego...


In [14]:
#Check the status of Truncated Post
lengths = list()
for sentence in df['Truncated_Post']:
    lengths.append(len(sentence.split()))
print(f"Maximum word sentence : {max(lengths)}")

Maximum word sentence : 101


In [15]:
fig = px.histogram(
    lengths,
    labels={'value':'Sequence length', 'count':'Counts'},
    title= "Counts of sequence lengths",
    text_auto = True,
)
fig.update_layout(bargap=0.2)
fig.show()

In [16]:
#Check the status of Cleaned Post
lengths_cleaned = list()
for sentence in df['clean_Post']:
    lengths_cleaned.append(len(sentence.split()))
print(f"Maximum word sentence : {max(lengths_cleaned)}")

Maximum word sentence : 4570


In [17]:
#It looks like choosing a max length of 350-400 words might work best
fig2 = px.histogram(
    lengths_cleaned,
    labels={'value':'Sequence length', 'count':'Counts'},
    title= "Counts of sequence lengths",
    text_auto = True,
)
fig2.update_layout(bargap=0.2)
fig2.show()

In [ ]:
#Create Mapping for ONET Name to ONET code
#Dict name2onet is common to both Train & Test data
name2onet = dict()
for row in df.itertuples():
  #print(row.ONET_NAME)
  if row.ONET_NAME not in name2onet:
    name2onet[row.ONET_NAME] = row.ONET

print(name2onet)

In [20]:
len(name2onet)

699

In [21]:
df['Truncated_Post'][:10]

0    executive meeting manager executive meeting ma...
1    rehabilitation technician outpatient rehab fle...
2    office bookkeeping assistant office bookkeepin...
3    administrative support coordinator va remote f...
4    receptionist administrative assistant receptio...
5    outsourced accounting manager zipapply outsour...
6    product owner you could be the one who changes...
7    extended coverage team specialist find jobs ex...
8    server waitstaff server waitstaff our place pi...
9    stockroom associate stockroom associate homego...
Name: Truncated_Post, dtype: object

In [22]:
#Initiate Sentence Transformer Model to determine similarities

model = SentenceTransformer('all-MiniLM-L6-v2')

#Embeddings for the ONET_NAME keys from dict->list, to be used in similarities
names = list(name2onet.keys())

embed_1net = model.encode(names, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [50]:
#Create: Truncated_Post & ONET_NAME embeddings & cos_sim of these
#Take top 10 Ranked suggestions

def similarities(text):

  #Embed the truncated post in text
  embed_post = model.encode(str(text), convert_to_tensor=True)
  #Match the above embedding for post with each ONET_NAME embedding - embed_lnet
  cos_sim = util.cos_sim(embed_post, embed_1net)

  #Convert the tensor above to np array
  cos_sim_np = cos_sim.numpy()[0]
  #Dict for attaching ONET_NAME to its similarity score
  sim_dict = {names[i]: cos_sim_np[i] for i in range(len(cos_sim_np))}

  #Sort the kv pairs from sim_dict in reverse & pick top 10
  top10 = sorted(sim_dict.items(), key=lambda kv:
                 (kv[1], kv[0]), reverse=True)[:10]

  return top10

In [43]:
#Create a list of TOP 10 predictions for Target ONET_NAME, w associated Ranks

df['top_10'] = df['Truncated_Post'].apply(lambda x : similarities(x))

In [27]:
#Fn onet_lst receives a list of 10 tuples: [(Onet_NAME, score)]*10
#Returns a list of top 10 ONET
def onet_lst(tup):
  return [name2onet[item[0]] for item in tup]

In [44]:

df['ONET_lst_10'] = df['top_10'].apply(lambda x : onet_lst(x))
#df[['top_10','ONET_NAME']].iloc[0]

In [45]:
#Add col 'Rank' to df, containing rank of True Label(ONET_NAME),if found in top_10 col for each row
#If NOT found, Rank contains 0

def found(df):
  present=[]

  print(df.shape)

  for row in df.itertuples():

    #Check if ONET_NAME exists in the top10 list created
    #row.top10 contains a list of 10 tuples: (Name, score)

    t10 = {item[0]: idx + 1 for idx, item in enumerate(row.top_10)}

    if row.ONET_NAME in t10.keys():

      present.append(t10[row.ONET_NAME])

    else:
      present.append(0)

  df['Rank'] = present
  return present

In [46]:
present = found(df)
#Number of Names missed in Top 10
miss = present.count(0)
acc = (len(present) - miss)/len(present)
print("Missed:", miss, "; Model Accuracy:", acc)

(17927, 9)
Missed: 7394 ; Model Accuracy: 0.5875495063312323


In [47]:
inv_rank = [1/x for x in present if x!=0]

mrr = sum(inv_rank)/len(present)
print("MRR:", mrr)

MRR: 0.36977935649073784


In [48]:
#df.drop(['ID', "POSTED", "ONET_lst"], axis=1, inplace=True)
df.head(10)

,TITLE_RAW,BODY,ONET_NAME,ONET,clean_Post,Truncated_Post,top_10,Rank,ONET_lst_10
0,Executive Meeting Manager,Executive Meeting Manager Marriott La Jolla - ...,"Meeting, Convention, and Event Planners",13-1121.00,executive meeting manager executive meeting ma...,executive meeting manager executive meeting ma...,"[(Hotel, Motel, and Resort Desk Clerks, 0.5007...",0,"[43-4081.00, 11-9081.00, 35-9031.00, 11-9051.0..."
1,Rehabilitation Technician-Outpatient Rehab-Fle...,Rehabilitation Technician-Outpatient Rehab-Fle...,Occupational Therapy Aides,31-2012.00,rehabilitation technician outpatient rehab fle...,rehabilitation technician outpatient rehab fle...,"[(Rehabilitation Counselors, 0.51282257), (Phy...",7,"[21-1015.00, 31-2022.00, 29-1123.00, 49-3092.0..."
2,Office/Bookkeeping Assistant,"Office/Bookkeeping Assistant\nSanta Barbara, C...","Office Clerks, General",43-9061.00,office bookkeeping assistant office bookkeepin...,office bookkeeping assistant office bookkeepin...,"[(Payroll and Timekeeping Clerks, 0.5860446), ...",0,"[43-3051.00, 43-3031.00, 43-4161.00, 41-2021.0..."
3,Administrative Support Coordinator - VA - (REM...,Find Jobs Administrative Support Coordinator -...,"Secretaries and Administrative Assistants, Exc...",43-6014.00,administrative support coordinator va remote f...,administrative support coordinator va remote f...,"[(Human Resources Specialists, 0.50527346), (O...",0,"[13-1071.00, 43-9199.00, 43-1011.00, 43-4161.0..."
4,Receptionist/Administrative Assistant,Receptionist/Administrative Assistant Burgess ...,"Secretaries and Administrative Assistants, Exc...",43-6014.00,receptionist administrative assistant receptio...,receptionist administrative assistant receptio...,"[(Receptionists and Information Clerks, 0.5929...",5,"[43-4171.00, 43-4161.00, 43-4121.00, 43-6012.0..."
5,Outsourced Accounting Manager,ZipApply\nOutsourced Accounting Manager\nGloba...,Financial Managers,11-3031.00,outsourced accounting manager zipapply outsour...,outsourced accounting manager zipapply outsour...,"[(Bookkeeping, Accounting, and Auditing Clerks...",7,"[43-3031.00, 13-1199.00, 13-2011.00, 41-3091.0..."
6,Product Owner,You could be the one who changes everything fo...,General and Operations Managers,11-1021.00,product owner you could be the one who changes...,product owner you could be the one who changes...,"[(Sales Representatives, Wholesale and Manufac...",0,"[41-4012.00, 41-4011.00, 27-1029.00, 27-1021.0..."
7,Extended Coverage Team Specialist,Find Jobs Extended Coverage Team Specialist Sw...,Fast Food and Counter Workers,35-3023.00,extended coverage team specialist find jobs ex...,extended coverage team specialist find jobs ex...,"[(Traffic Technicians, 0.44290614), (Compensat...",0,"[53-6041.00, 13-1141.00, 47-4051.00, 17-3029.0..."
8,Server/Waitstaff,"Server/Waitstaff Our Place Pizzeria Branford, ...",Waiters and Waitresses,35-3031.00,server waitstaff server waitstaff our place pi...,server waitstaff server waitstaff our place pi...,"[(Food Servers, Nonrestaurant, 0.5792421), (Wa...",2,"[35-3041.00, 35-3031.00, 11-9051.00, 35-1012.0..."
9,Stockroom Associate,Stockroom Associate HomeGoods (part of The TJX...,"Laborers and Freight, Stock, and Material Move...",53-7062.00,stockroom associate stockroom associate homego...,stockroom associate stockroom associate homego...,"[(Sales and Related Workers, All Other, 0.4476...",0,"[41-9099.00, 53-3031.00, 43-3051.00, 41-2022.0..."


In [ ]:
df['top_10'].iloc[25]

[('Health and Safety Engineers, Except Mining Safety Engineers and Inspectors',
  0.47885364),
 ('Explosives Workers, Ordnance Handling Experts, and Blasters', 0.4673443),
 ('Manufacturing Engineers', 0.45049918),
 ('Occupational Health and Safety Specialists', 0.43795806),
 ('Airfield Operations Specialists', 0.42444608),
 ('Occupational Health and Safety Technicians', 0.42394903),
 ('Industrial Engineers', 0.4197712),
 ('Millwrights', 0.4141938),
 ('Compensation, Benefits, and Job Analysis Specialists', 0.39749354),
 ('Industrial Truck and Tractor Operators', 0.39459574)]

In [ ]:
df['ONET_lst'].iloc[25]

['17-2111.00',
 '47-5032.00',
 '17-2112.03',
 '19-5011.00',
 '53-2022.00',
 '19-5012.00',
 '17-2112.00',
 '49-9044.00',
 '13-1141.00',
 '53-7051.00']

In [ ]:
#Find Max length of sentence for the Model
print("Max Sequence Length:", model.max_seq_length)

Max Sequence Length: 256


In [51]:
df_test.shape

(19394, 8)

In [79]:
#Predict the TOP 10 ONET list for TEST data

df_test['top_10'] = df_test['Truncated_Post'].apply(lambda x : similarities(x))

df_test['ONET_lst_10'] = df_test['top_10'].apply(lambda x : onet_lst(x))

present_test = found(df_test)

#Number of Names missed in Top 10
miss = present_test.count(0)
acc = (len(present_test) - miss)/len(present_test)
print("Missed:", miss, "; Model Accuracy:", acc)

(19394, 8)
Missed: 8002 ; Model Accuracy: 0.5873981643807363


In [80]:
#sentence_transformers.evaluation.InformationRetrievalEvaluator; Mean Reciprocal Rank (MRR), Recall@k

inv_rank_test = [1/x for x in present_test if x!=0]

mrr = sum(inv_rank_test)/len(present_test)
print("MRR:", mrr)

MRR: 0.36423775312606477


In [82]:
df_test.head(10)

,TITLE_RAW,BODY,ONET_NAME,ONET,clean_Post,Truncated_Post,top_10,ONET_lst_10,Rank
0,Grocery Order Writer (Buyer / Inventory Replen...,Grocery Order Writer (Buyer / Inventory Replen...,"Purchasing Agents, Except Wholesale, Retail, a...",13-1023.00,grocery order writer buyer inventory replenish...,grocery order writer buyer inventory replenish...,"[(Shipping, Receiving, and Inventory Clerks, 0...","[43-5071.00, 35-1012.00, 11-9051.00, 41-1012.0...",0
1,Superintendent,Apply to this job. \nThink you're the perfect ...,"Education Administrators, Kindergarten through...",11-9032.00,superintendent apply to this job think you re ...,superintendent apply to this job think you re ...,"[(Construction Managers, 0.4469341), (Business...","[11-9021.00, 13-1199.00, 11-1021.00, 11-9199.0...",0
2,Software Developer IV,Software Developer IV\nJob Locations\nUS-NE-Om...,Software Developers,15-1252.00,software developer iv software developer iv jo...,software developer iv software developer iv jo...,"[(Software Developers, 0.49736357), (Sales Eng...","[15-1252.00, 41-9031.00, 15-1254.00, 17-2112.0...",1
3,Auto Glass Technician,Auto Glass Technician Gerber Collision & Glass...,Automotive Service Technicians and Mechanics,49-3023.00,auto glass technician auto glass technician ge...,auto glass technician auto glass technician ge...,"[(Automotive Glass Installers and Repairers, 0...","[49-3022.00, 49-3023.00, 53-6041.00, 13-1141.0...",2
4,Food and Beverage Operations Manager,Food and Beverage Operations Manager Wavetroni...,Food Service Managers,11-9051.00,food and beverage operations manager food and ...,food and beverage operations manager food and ...,"[(Food Service Managers, 0.5846275), (First-Li...","[11-9051.00, 35-1012.00, 51-3092.00, 35-9031.0...",1
5,Retail Sales Specialist (Bilingual Spanish),Retail Sales Specialist (Bilingual Spanish) Sp...,Retail Salespersons,41-2031.00,retail sales specialist bilingual spanish reta...,retail sales specialist bilingual spanish reta...,"[(Retail Salespersons, 0.40428388), (Sales Eng...","[41-2031.00, 41-9031.00, 41-4012.00, 41-4011.0...",1
6,SQL Integration Specialist,Apply to this job. \nThink you're the perfect ...,Software Developers,15-1252.00,sql integration specialist apply to this job t...,sql integration specialist apply to this job t...,"[(Data Warehousing Specialists, 0.6452333), (B...","[15-1243.01, 13-1199.00, 15-2051.01, 15-1242.0...",0
7,Experienced Veterinarian Technician (FT),Experienced Veterinarian Technician (FT) Lakes...,Veterinarians,29-1131.00,experienced veterinarian technician ft experie...,experienced veterinarian technician ft experie...,"[(Veterinary Technologists and Technicians, 0....","[29-2056.00, 29-1131.00, 31-9096.00, 19-4021.0...",2
8,Production Worker/Machine Operator,Production Worker/Machine Operator Focus Workf...,Helpers--Production Workers,51-9198.00,production worker machine operator production ...,production worker machine operator production ...,[(Operating Engineers and Other Construction E...,"[47-2073.00, 49-9043.00, 51-1011.00, 53-7051.0...",0
9,RBT (Registered Behavior Technician),RBT (Registered Behavior Technician) Thrive Au...,Registered Nurses,29-1141.00,rbt registered behavior technician rbt registe...,rbt registered behavior technician rbt registe...,"[(Training and Development Specialists, 0.4651...","[13-1151.00, 21-1099.00, 31-2012.00, 31-2011.0...",0


In [88]:
#Given a Job Posting, in text form, the Fn returns a list of top 10 ONET codes in a ranked list
#The input can be either the Raw posting, title or concatenation of both

def predict(post):
  clean = clean_text(post)
  truncated = trunc(clean)
  top_10 =  similarities(truncated)
  ONET_lst_10 = onet_lst(top_10)

  return ONET_lst_10


In [92]:
print(predict(df_test['Truncated_Post'].iloc[2]))

['15-1252.00', '41-9031.00', '15-1254.00', '17-2112.00', '13-1151.00', '15-1299.08', '17-2112.03', '17-3023.00', '17-3026.00', '15-1251.00']


In [93]:
print(df_test['ONET'].iloc[2])

15-1252.00
